In [3]:
!pip install jupyter-dash dash pandas plotly

Defaulting to user installation because normal site-packages is not writeable


In [23]:
# Import required libraries
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import nest_asyncio
from IPython.display import display

In [29]:
# Apply workaround to allow nested event loops in Jupyter
nest_asyncio.apply()

# Read dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create Dash app (standard way)
app = Dash(__name__)

In [30]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [44]:
# App layout
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')])
def update_scatter_plot(entered_site, payload_range):
    low, high = payload_range
    # Filter the data based on payload range
    mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
    filtered_df = spacex_df[mask]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title='Correlation Between Payload and Success for All Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         title=f'Correlation Between Payload and Success for site {entered_site}')
    return fig

In [45]:
# Callback for pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value'))
def update_pie_chart(entered_site):
    if entered_site == 'ALL':
        site_success = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(site_success, names='Launch Site',
                     title='Total Success Launches for All Sites')
    else:
        site_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        site_df = site_df.groupby('class').size().reset_index(name='class count')
        fig = px.pie(site_df, values='class count', names='class',
                     title=f"Success vs Failure for site {entered_site}",
                     color='class', color_discrete_map={1: 'green', 0: 'red'})
    return fig

In [46]:
# Callback for scatter chart
dcc.RangeSlider(id='payload-slider',
    min=spacex_df['Payload Mass (kg)'].min(),
    max=spacex_df['Payload Mass (kg)'].max(),
    step=1000,
    marks={i: f'{i}' for i in range(0, 10001, 2500)},
    value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()])

RangeSlider(min=0.0, max=9600.0, step=1000, marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}, value=[0.0, 9600.0], id='payload-slider')

In [47]:
# Run the app inside the notebook
app.run(mode='inline')

In [40]:
print(spacex_df.head())
print(spacex_df.columns)

   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  
Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')
